In [ ]:
import sys
import os
import matplotlib.ticker as ticker
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
%matplotlib inline

In [ ]:
# parameters
# casename = 'COREII_Global'
# casename = 'JRA55-do_Global'
casename = 'JRA55-do_Global_dampV5d'
var = 'mld_deltaR_mean'
# var = 'PE_delta'
# var = 'SST_mean'
plot_figure = True

In [ ]:
# paths
s2data_root = os.environ['GOTMFIG_ROOT']+'/data/'+casename
fig_root = os.environ['GOTMFIG_ROOT']+'/'+casename
os.makedirs(s2data_root, exist_ok=True)
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# lists
timetag_list = ['20090101-20090131',
                '20090201-20090228',
                '20090301-20090331',
                '20090401-20090430',
                '20090501-20090531',
                '20080601-20080630',
                '20080701-20080731',
                '20080801-20080831',
                '20080901-20080930',
                '20081001-20081031',
                '20081101-20081130',
                '20081201-20081231']
ntag = len(timetag_list)
turbmethod_list = ['KPP-CVMix',
                   'KPP-ROMS',
                   'EPBL',
                   'SMC',
                   'K-EPSILON-SG',
                   'KPPLT-EFACTOR',
                   'KPPLT-ENTR',
                   'KPPLT-RWHGK',
                   'EPBL-LT',
                   'SMCLT',
                   'OSMOSIS']
legend_list = ['KPP-CVMix',
               'KPP-ROMS',
               'ePBL',
               'SMC-KC94',
               'k-epsilon',
               'KPPLT-VR12',
               'KPPLT-LF17',
               'KPPLT-R16',
               'ePBL-LT',
               'SMCLT-H15',
               'OSMOSIS']
nm = len(turbmethod_list)
bcolor = ['lightcyan','lightblue','lightgreen','lightpink','palegoldenrod',
          'turquoise', 'deepskyblue','royalblue','forestgreen','firebrick','mediumpurple']
month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
indices = [0, 1, 2, 3, 4, 5, 6, 8, 9]
nidx = len(indices)

In [ ]:
# get data and save in a two dimensional list
gmobj_data_arr = []
for j in np.arange(ntag):
    timetag = timetag_list[j]
    # read data
    gmobj_data_tmp = []
    for i in np.arange(nm):
        tmname = turbmethod_list[i]
        s2data_name = s2data_root+'/VR1m_DT600s_'+timetag+'/data_'+var+'_'+tmname+'.npz'
        gmobj_tmp = GOTMMap().load(s2data_name)
        gmobj_data_tmp.append(gmobj_tmp)
    gmobj_data_arr.append(gmobj_data_tmp)

In [ ]:
# median or mean (over all non-Langmuir turbulent methods) of var at each month, [12]*GOTMMap
gmobj_m_mon_std_r = []
gmobj_m_mon_std_v = []
# loop over time tags
for j in np.arange(ntag):
    ncase = gmobj_data_arr[j][0].data.size
    tmp = np.zeros([nm, ncase])
    # loop over turbulent methods
    for i in np.arange(nm):
        tmp[i,:] = gmobj_data_arr[j][i].data
    # mean of all non-Langmuir cases
    diag0_nlt = np.mean(tmp[0:5,:], axis=0)
    tmpr = np.zeros([nidx, ncase])
    tmpv = np.zeros([nidx, ncase])
    for i in np.arange(nidx):
        idx = indices[i]
        tmpr[i,:] = tmp[idx,:]/diag0_nlt
        tmpv[i,:] = tmp[idx,:]
    diag1_std_r = np.std(tmpr, axis=0)
    diag1_std_v = np.std(tmpv, axis=0)
    # get lon, lat etc.
    lon = gmobj_data_arr[j][0].lon
    lat = gmobj_data_arr[j][0].lat
    name = gmobj_data_arr[j][0].name
    units = gmobj_data_arr[j][0].units
    # create GOTMMap object
    gmobj_tmp = GOTMMap(data=diag1_std_r, lon=lon, lat=lat, name=name, units=units)
    # append to list
    gmobj_m_mon_std_r.append(gmobj_tmp)
    # create GOTMMap object
    gmobj_tmp = GOTMMap(data=diag1_std_v, lon=lon, lat=lat, name=name, units=units)
    # append to list
    gmobj_m_mon_std_v.append(gmobj_tmp)

In [ ]:
# plot figure
if plot_figure:
    # create figure
    nrow = 6
    fig_width = 12
    fig_height = 13

    # plot figure
    height_ratios = [1]*nrow
    height_ratios.append(0.15)
    width_ratios = [1, 1, 0.05]
    f, axarr = plt.subplots(nrow, 2, sharex='col')
    f.set_size_inches(fig_width, fig_height)
    
    # plot
    levels = np.arange(41)*0.5
    for j in np.arange(12):
        m = j//2
        n = j%2
        mlabel = month_labels[j]
        im0 = gmobj_m_mon_std_v[j].plot(axis=axarr[m,n], levels=levels, add_colorbar=False)
        # label
        axarr[m,n].text(40, 55, mlabel, fontsize=14, color='k', fontweight='bold', va='top',
                bbox=dict(boxstyle="square",ec='k',fc='w'))
        
    # reduce margin
    plt.tight_layout()
    
    # colorbar
    plt.subplots_adjust(right=0.95)
    cax0 = plt.axes([0.85, 0.15, 0.1, 0.7])
    cax0.set_visible(False)
    cb0 = plt.colorbar(im0, ax=cax0, aspect=35)
    
    # save figure
    figname = fig_root+'/fig_std_all_'+var+'.png'
    plt.savefig(figname, dpi = 300)

In [ ]:
# plot figure: R
if plot_figure:
    # create figure
    nrow = 6
    fig_width = 12
    fig_height = 13

    # plot figure
    height_ratios = [1]*nrow
    height_ratios.append(0.15)
    width_ratios = [1, 1, 0.05]
    f, axarr = plt.subplots(nrow, 2, sharex='col')
    f.set_size_inches(fig_width, fig_height)
    
    # plot
    levels = np.arange(33)*0.005
    for j in np.arange(12):
        m = j//2
        n = j%2
        mlabel = month_labels[j]
        im0 = gmobj_m_mon_std_r[j].plot(axis=axarr[m,n], levels=levels, add_colorbar=False)
        # label
        axarr[m,n].text(40, 55, mlabel, fontsize=14, color='k', fontweight='bold', va='top',
                bbox=dict(boxstyle="square",ec='k',fc='w'))
        
    # reduce margin
    plt.tight_layout()
    
    # colorbar
    plt.subplots_adjust(right=0.95)
    cax0 = plt.axes([0.85, 0.15, 0.1, 0.7])
    cax0.set_visible(False)
    cb0 = plt.colorbar(im0, ax=cax0, aspect=35)
    
    # save figure
    figname = fig_root+'/fig_std_r_all_'+var+'.png'
    plt.savefig(figname, dpi = 300)